# Sentiment Analysis with BERT

In this notebook, we will perform sentiment analysis using BERT (Bidirectional Encoder Representations from Transformers). Sentiment analysis is the task of classifying the sentiment expressed in a piece of text, typically as positive, negative, or neutral.
<p align="center">
  <img src="https://www.researchgate.net/publication/352081817/figure/fig3/AS:1030485023154177@1622698183794/Workflow-of-BERT-Sentiment-Analysis-detailing-Self-Attention.ppm" alt="BERT Model" width="1500"/>
</p>

## Table of Contents
1. [Setup and Installation](#Setup)
2. [Data Loading and Exploration](#Data)
3. [Data Preprocessing](#Preprocessing)
4. [Model Building with BERT](#Model)
5. [Training the Model](#Training)
6. [Evaluating the Model](#Evaluation)
7. [Conclusion](#Conclusion)

# Setup and Installation

First, we need to install the necessary libraries. We'll be using `transformers` from Hugging Face for BERT, `torch` for deep learning, and `sklearn` for evaluation metrics.


In [ ]:
# Install required packages
!pip install transformers
!pip install torch
!pip install sklearn
!pip install pandas
!pip install matplotlib

In [ ]:
import torch
print(f"Using device: {'GPU' if torch.cuda.is_available() else 'CPU'}")

## Data Loading and Exploration

For this tutorial, we'll use the [IMDB Movie Reviews](https://ai.stanford.edu/~amaas/data/sentiment/) dataset, which contains 50,000 movie reviews labeled as positive or negative.

Let's load the data and take a quick look.


In [ ]:
import pandas as pd
# For demonstration, we'll use the `datasets` library
!pip install datasets

from datasets import load_dataset

# Load the IMDB dataset
dataset = load_dataset('imdb')

# Display the first few samples
print(dataset['train'][1403])

## Data Preprocessing

Before feeding the data into BERT, we need to preprocess it. This involves tokenization and encoding the text data into a format that BERT can understand.

We'll use the `BertTokenizer` for this purpose.


In [ ]:
from transformers import BertTokenizer

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define a function to tokenize the data
def tokenize_function(example):
    return tokenizer(example['text'], padding='max_length', truncation=True, max_length=512)

# Tokenize the datasets
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Rename the 'label' column to 'labels' as expected by the model
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

# Set the format for PyTorch
tokenized_datasets.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

# Split into train and test
train_dataset = tokenized_datasets['train']
test_dataset = tokenized_datasets['test']

## Model Building with BERT

We'll use the pre-trained `BertForSequenceClassification` model, which is suitable for classification tasks like sentiment analysis.

Let's initialize the model.


In [ ]:
from transformers import BertForSequenceClassification

# Initialize the model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Move the model to GPU if available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

## Training the Model

We'll use the `Trainer` API from Hugging Face for training. This simplifies the training loop and handles many aspects like evaluation and checkpointing.

Let's set up the training arguments and start training.


In [ ]:
from transformers import Trainer, TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,              # Number of training epochs
    per_device_train_batch_size=8,   # Batch size for training
    per_device_eval_batch_size=16,   # Batch size for evaluation
    warmup_steps=500,                # Number of warmup steps
    weight_decay=0.01,               # Strength of weight decay
    logging_dir='./logs',            # Directory for storing logs
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",                # Disable W&B integration
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Start training
trainer.train()

## Evaluating the Model

After training, we'll evaluate the model's performance on the test set. We'll calculate metrics such as accuracy, precision, recall, and F1-score.

Let's perform the evaluation.


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=1).numpy()
    labels = labels
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

# Re-initialize the Trainer with the compute_metrics function
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Evaluate the model
results = trainer.evaluate()

print(results)

### Confusion Matrix

To better understand the model's performance, let's plot the confusion matrix.


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import numpy as np

# Get predictions
predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=1)
labels = predictions.label_ids

# Compute confusion matrix
cm = confusion_matrix(labels, preds)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

## Demo

In this section, we'll demonstrate the trained BERT model's ability to predict the sentiment of various example sentences. This will provide a clear understanding of how the model performs on different inputs.


In [ ]:
# Define a list of example sentences for sentiment prediction
example_sentences = [
    "I absolutely loved this movie! The performances were outstanding.",
    "This was the worst film I have ever seen. Completely disappointing.",
    "The plot was intriguing, but the characters were underdeveloped.",
    "An average movie with some good moments and some bad ones.",
    "I was not impressed by the storyline or the acting.",
    "Fantastic visuals and a gripping story. Highly recommend!",
    "The movie was okay, nothing special but not terrible either.",
    "I didn't enjoy this movie at all. It was a waste of time.",
    "A masterpiece of modern cinema. Truly inspiring.",
    "Mediocre at best. I expected much more from this director."
]

In [ ]:
def predict_sentiment(sentence):
    # Tokenize the input sentence
    inputs = tokenizer.encode_plus(
        sentence,
        add_special_tokens=True,
        max_length=512,
        truncation=True,
        padding='max_length',
        return_tensors='pt'
    )

    # Move tensors to the appropriate device
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    # Disable gradient calculation for inference
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    predicted_class_id = logits.argmax().item()

    # Map the predicted class to label
    label = 'Positive' if predicted_class_id == 1 else 'Negative'
    return label

In [ ]:
import pandas as pd

# Create a DataFrame to display sentences and their predicted sentiments
demo_results = []

for sentence in example_sentences:
    prediction = predict_sentiment(sentence)
    demo_results.append({'Sentence': sentence, 'Predicted Sentiment': prediction})

df_demo = pd.DataFrame(demo_results)
df_demo

### Results in a table
| Example Sentence                                                                                          | Predicted Sentiment |
|-----------------------------------------------------------------------------------------------------------|---------------------|
| I absolutely loved this movie! The performances were outstanding.                                         | Positive            |
| This was the worst film I have ever seen. Completely disappointing.                                      | Negative            |
| The plot was intriguing, but the characters were underdeveloped.                                         | Negative            |
| An average movie with some good moments and some bad ones.                                               | Negative            |
| I was not impressed by the storyline or the acting.                                                      | Negative            |
| Fantastic visuals and a gripping story. Highly recommend!                                               | Positive            |
| The movie was okay, nothing special but not terrible either.                                             | Negative            |
| I didn't enjoy this movie at all. It was a waste of time.                                                | Negative            |
| A masterpiece of modern cinema. Truly inspiring.                                                         | Positive            |
| Mediocre at best. I expected much more from this director.                                              | Negative            |